# Installing deps

In [1]:
pip install --quiet -U langchain langchain_community tiktoken langchain-ollama bs4 scikit-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Users/illiapuzanov/personal/llm/dodo/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Embedding - GPT4ll Embeddings

`pip install langchain-nomic "nomic[local]"`

UPD: gpt4all doesn't work on intel chip macs...

Will use `OllamaEmbeddings` instead.

### LLM

We are using Ollama with llama3.2:

`ollama pull llama3.2:3b-instruct-fp16`

### Tooling

[Tavily](https://tavily.com) for web search, optimised for LLMs and RAG

[LangSmith](https://smith.langchain.com) for LLM tracing


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'llmdoc'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

# Init Search
from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

In [3]:
# LLM
from langchain_ollama import ChatOllama
local_llm = 'llama3.2:3b-instruct-fp16'
llm = ChatOllama(model=local_llm, temperature=0)
llm_json_mode = ChatOllama(model=local_llm, temperature=0, format='json')

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import OllamaEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    #"https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    #"https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"
]

# Load document
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split documents
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200
)
doc_splits = text_splitter.split_documents(docs_list)

In [5]:
# Add to vector DB
vectorstore = SKLearnVectorStore.from_documents(
    documents=doc_splits,
    embedding=OllamaEmbeddings(model=local_llm),
)

retriever = vectorstore.as_retriever(k=3)

In [6]:
retriever.batch(["what is llm?"])

[[Document(metadata={'id': '58402a41-14c7-4560-8585-9aece071bca1', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en'}, page_content='}\n]\nChallenges#\nAfter going through key ideas and demos of building LLM-centered agents, I start to see a couple common limitations:'),
  Document(metadata={'id': '6fc845ab-c7ee-4ce9-bb23-ef8c69460bc0', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 